In [18]:
import csv
import pandas as pd
import clip
import torch
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from image_database import *
from PIL import Image
from scipy.spatial.distance import cdist
import bisect 
from scipy.spatial import ConvexHull
import gc
import debias_clip as dclip
#import cvxpy as cp
image_folder_prefix = 'datasets/celeba/img_align_celeba/'

data = pd.read_csv('datasets/celeba/alg_testing.csv') #datasets\occuptations_2\occupations_labels.csv
data = data.astype('string')


In [ ]:
print(data[data['Pale_Skin'] == '1'].head())

for image_loc in data[data['Pale_Skin'] == '-1']['image_id']:
    image = Image.open(image_folder_prefix + image_loc)
    plt.imshow(image)
    plt.show()

In [19]:
features = np.load('datasets/celeba/features.npy')[20000:35000]

features_debias = np.load('datasets/celeba/features_debias.npy')[20000:35000]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)





In [20]:
device_d = 'cpu'
model_debias, preprocess_debias = dclip.load("ViT-B/16-gender", device_d)

Installing pretrained embedings
 best_ndkl_oai-clip-vit-b-16_neptune_run_OXVLB-317_model_e4_step_5334_embeddings.pt...


100%|█████████████████████████████████████| 4.73k/4.73k [00:00<00:00, 1.23MiB/s]


In [5]:
print(data['5_o_Clock_Shadow'])
data.head()



0        -1
1        -1
2        -1
3        -1
4         1
         ..
14995    -1
14996    -1
14997    -1
14998    -1
14999    -1
Name: 5_o_Clock_Shadow, Length: 15000, dtype: string


,Unnamed: 0,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,20000,020001.jpg,-1,1,1,1,-1,-1,-1,1,...,-1,-1,-1,-1,1,-1,1,1,-1,1
1,20001,020002.jpg,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,-1
2,20002,020003.jpg,-1,-1,1,-1,-1,1,-1,-1,...,-1,-1,1,-1,1,-1,-1,-1,-1,1
3,20003,020004.jpg,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
4,20004,020005.jpg,1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,1


In [22]:
catagories = ['attractive person', 'blurry photo of a person', 'person with brown hair', 'person wearing eyeglasses', 'person with gray hair', 'smiling person', 'person in a hat']
catagory_column = ['Attractive', 'Blurry', 'Brown_Hair', 'Eyeglasses', 'Gray_Hair', 'Smiling', 'Wearing_Hat']

image_database = ImageDatabase(features, data, model, preprocess, device)
indistinguisable_values = [["1"] for cat in catagory_column]
totals_by_cat = {cat: len(data[data[catagory_column[i]] == "1"]) for i, cat in enumerate(catagories)}
image_database.define_coordinate_mapping(['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']])

debias_database = ImageDatabase(features_debias, data, model_debias, preprocess_debias, device_d)

image_database.add_clipclip_ordering("gender", np.load('datasets/MI_orders/gender.npy'))
image_database.add_clipclip_ordering("skintone", np.load('datasets/MI_orders/skintone.npy'))
image_database.add_clipclip_ordering("age", np.load('datasets/MI_orders/age.npy'))
image_database.add_clipclip_ordering("gender_skintone", np.load('datasets/MI_orders/gender_skintone.npy'))
image_database.add_clipclip_ordering("intersectional", np.load('datasets/MI_orders/intersectional.npy'))
image_database.add_clipclip_ordering("additional_concepts", np.load('datasets/MI_orders/additional_concepts.npy'))



method_name_specification_list = [
    (lambda k, tol: lambda x: image_database.search(x, k), 'Baseline', []),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_gender', [("sensitive_attributes", [("A picture of a man", "A picture of a woman")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_gender_skintone', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_skintone', [("sensitive_attributes", [("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_age', [("sensitive_attributes", [("A picture of an old person", "A picture of a young person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person"), ("A picture of an old person", "A picture of a young person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_4_attr', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person"), ("A picture of an old person", "A picture of a young person"), ("A picture of an American person", "A picture of a non-American person")])]),

    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_gender', [("sensitive_attributes", [("A picture of a man", "A picture of a woman")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_gender_skintone', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_skintone', [("sensitive_attributes", [("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_age', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person"), ("A picture of an old person", "A picture of a young person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_4_attr', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person"), ("A picture of an old person", "A picture of a young person"), ("A picture of an American person", "A picture of a non-American person")])]),

    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='feature_distances'), 'CDI_Features', []),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='true_labels'), 'CDI_TrueConcept', []),
    
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_gender', [("pbm_classes", ["unknown gender", "man", "woman"])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_intersectional', [("pbm_classes", ["unknown gender and skin-tone", "light-skinned man", "light-skinned woman", "dark-skinned man", "dark-skinned woman"])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_skintone', [("pbm_classes", ["unknown skin-tone", "light-skinned person", "dark-skinned person"])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_three_attributes', [("pbm_classes", intersections_for_pbm([["light-skinned", "dark-skinned"], ["old", "young"], ["man", "woman"]], "an unknown skin-tone, age, and gender", ""))]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_four_attributes', [("pbm_classes", intersections_for_pbm([["light-skinned", "dark-skinned"], ["old", "young"], ["American", "non-American"], ["man", "woman"]], "an unknown skin-tone, age, nationality, and gender", ""))]),
    
    (lambda k, n: lambda x: image_database.clip_clip(x, "gender", n, k), 'CLIP_gender', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "skintone", n, k), 'CLIP_skintone', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "age", n, k), 'CLIP_age', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "gender_skintone", n, k), 'CLIP_gender_skintone', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "intersectional", n, k), 'CLIP_intersectional', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "additional_concepts", n, k), 'CLIP_add_concept', []),

    (lambda k, tol: lambda x: debias_database.search(x, k), 'DebiasClip', [])
]

method_name_specification_list = [
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='random'), 'CDI_Random', [("sensitive_attributes", [("A picture of a man", "A picture of a woman")])]),
]



ks = [10, 25, 50, 100]  # [10, 25, 50, 100] 

number_of_tol_steps = 16 # 16
number_of_eps_steps = 11 # 11
number_of_clip_clip_steps = 24 #24
random_iters = 4 #4

result_dicts = []

for method, name, spec in method_name_specification_list:
    print(f"Starting analysis for method: {name}...")
    for s, val in spec:
        if s == "sensitive_attributes":
            image_database.sensitive_attributes(val)
        if s == "pbm_classes":
            image_database.define_pbm_classes(val)
    for k in tqdm(ks):
        result_dict = {'name': name}

        if name in ['Baseline', "DebiasClip"]:
            steps = 1
        else:
            steps = number_of_tol_steps

        if name[0:3] == 'PBM':
            for e in reversed(range(0, number_of_eps_steps)):
                eps = e / (number_of_eps_steps - 1)
                retrieval_function = method(k, eps)
                new_dict = result_dict.copy()
                random_results = []
                for i in range(random_iters):
                    new_dict = result_dict.copy()
                    run_analysis_celeba(retrieval_function, k, eps, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                    random_results.append(new_dict)
                
                add_dict = result_dict.copy()
                for key in random_results[0].keys():
                    if key == 'name':
                        continue
                    add_dict[key] = np.mean([res[key] for res in random_results], axis=0)
                result_dicts.append(add_dict)

        elif name[0:4] == 'CLIP':
            for e in range(0, number_of_clip_clip_steps):
                n = e * 20
                retrieval_function = method(k, n)
                new_dict = result_dict.copy()
                run_analysis_celeba(retrieval_function, k, n, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                result_dicts.append(new_dict)
        else:
            for t in range(0, steps):
                if steps == 1:
                    tol = None
                else:
                    tol = t / 200
                retrieval_function = method(k, tol)
                new_dict = result_dict.copy()
                if name == "CDI_Random":
                    random_results = []
                    for i in range(random_iters):
                        new_dict = result_dict.copy()
                        run_analysis_celeba(retrieval_function, k, tol, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                        random_results.append(new_dict)
                    new_dict = result_dict.copy()

                    for key in random_results[0].keys():
                        if key == 'name':
                            continue
                        new_dict[key] = np.mean([res[key] for res in random_results], axis=0)
                else:
                    run_analysis_celeba(retrieval_function, k, tol, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                result_dicts.append(new_dict)

df = parse_analysis_celeba(result_dicts, ['Pale_Skin', 'Male', 'Young'])

print(df)

df.to_pickle("results/celeba-random.pkl")
        

Starting analysis for method: CDI_Random...


100%|██████████| 4/4 [1:18:30<00:00, 1177.74s/it]

                     method        name    tol      k  Avg_Precision  \
0     CDI_Random (tol: 0.0)  CDI_Random  0.000   10.0       0.800000   
1   CDI_Random (tol: 0.005)  CDI_Random  0.005   10.0       0.753571   
2    CDI_Random (tol: 0.01)  CDI_Random  0.010   10.0       0.778571   
3   CDI_Random (tol: 0.015)  CDI_Random  0.015   10.0       0.757143   
4    CDI_Random (tol: 0.02)  CDI_Random  0.020   10.0       0.703571   
..                      ...         ...    ...    ...            ...   
59  CDI_Random (tol: 0.055)  CDI_Random  0.055  100.0       0.545357   
60   CDI_Random (tol: 0.06)  CDI_Random  0.060  100.0       0.544643   
61  CDI_Random (tol: 0.065)  CDI_Random  0.065  100.0       0.536786   
62   CDI_Random (tol: 0.07)  CDI_Random  0.070  100.0       0.533571   
63  CDI_Random (tol: 0.075)  CDI_Random  0.075  100.0       0.538929   

    Avg_Recall  Avg_PutI  Avg_AbsBias_Pale_Skin  Avg_Bias_Pale_Skin  \
0     0.006960  0.800000               0.942857           -0.942

In [7]:
df = pd.read_pickle("results/celeba-3-24.pkl")



0.7


In [12]:

method_name_specification_list = [
    (lambda k, tol: lambda x: image_database.search(x, k), 'Baseline', []),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a white person", "A picture of a non-white person"), ("A picture of an old person", "A picture of a young person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a white person", "A picture of a non-white person"), ("A picture of an old person", "A picture of a young person")])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_three_attributes', [("pbm_classes", intersections_for_pbm([["white", "non-white"], ["old", "young"], ["man", "woman"]], "an unknown skin-tone, age, and gender", ""))]),
    (lambda k, n: lambda x: image_database.clip_clip(x, "intersectional", n, k), 'CLIP_intersectional', []),
    (lambda k, tol: lambda x: debias_database.search(x, k), 'DebiasClip', [])
]
np.random.seed(123)

catagories = ['attractive person', 'blurry photo of a person', 'person with brown hair', 'person wearing eyeglasses', 'person with gray hair', 'smiling person', 'person in a hat']
catagory_column = ['Attractive', 'Blurry', 'Brown_Hair', 'Eyeglasses', 'Gray_Hair', 'Smiling', 'Wearing_Hat']

features = np.load('datasets/celeba/features.npy')
features_debias = np.load('datasets/celeba/features_debias.npy')

data = pd.read_csv('datasets/celeba/list_attr_celeba.csv')
data = data.astype('string')

start_idx = 20000
step_idx = 15000

result_dicts = []
for runs in range(5):
    lb = start_idx + runs * step_idx
    ub = start_idx + (runs + 1) * step_idx
    f_random = features[lb:ub]
    d_random = data.iloc[lb:ub]
    fd_random = features_debias[lb:ub]
    image_database = ImageDatabase(f_random, d_random, model, preprocess, device)
    indistinguisable_values = [["1"] for cat in catagory_column]
    totals_by_cat = {cat: len(data[data[catagory_column[i]] == "1"]) for i, cat in enumerate(catagories)}
    image_database.define_coordinate_mapping(['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']])
    
    debias_database = ImageDatabase(fd_random, d_random, model_debias, preprocess_debias, device_d)
    image_database.add_clipclip_ordering("intersectional", np.load('datasets/MI_orders/intersectional.npy'))

    ks = [10, 25, 50, 100] # [10, 25, 50, 100] 

    number_of_tol_steps = 16 # 16
    number_of_eps_steps = 11 # 11
    number_of_clip_clip_steps = 24 #24
    random_iters = 4 #4

    for method, name, spec in method_name_specification_list:
        print(f"Starting analysis for method: {name}...")
        for s, val in spec:
            if s == "sensitive_attributes":
                image_database.sensitive_attributes(val)
            if s == "pbm_classes":
                image_database.define_pbm_classes(val)
        for k in tqdm(ks):
            result_dict = {'name': name}

            if name in ['Baseline', "DebiasClip"]:
                steps = 1
            else:
                steps = number_of_tol_steps

            if name[0:3] == 'PBM':
                for e in reversed(range(0, number_of_eps_steps)):
                    eps = e / (number_of_eps_steps - 1)
                    retrieval_function = method(k, eps)
                    new_dict = result_dict.copy()
                    random_results = []
                    for i in range(random_iters):
                        new_dict = result_dict.copy()
                        run_analysis_celeba(retrieval_function, k, eps, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                        random_results.append(new_dict)
                    
                    add_dict = result_dict.copy()
                    for key in random_results[0].keys():
                        if key == 'name':
                            continue
                        add_dict[key] = np.mean([res[key] for res in random_results], axis=0)
                    result_dicts.append(add_dict)

            elif name[0:4] == 'CLIP':
                for e in range(0, number_of_clip_clip_steps):
                    n = e * 20
                    retrieval_function = method(k, n)
                    new_dict = result_dict.copy()
                    run_analysis_celeba(retrieval_function, k, n, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                    result_dicts.append(new_dict)
            else:
                for t in range(0, steps):
                    if steps == 1:
                        tol = None
                    else:
                        tol = t / 200
                    retrieval_function = method(k, tol)
                    new_dict = result_dict.copy()
                    if name == "CDI_Random":
                        random_results = []
                        for i in range(random_iters):
                            new_dict = result_dict.copy()
                            run_analysis_celeba(retrieval_function, k, tol, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                            random_results.append(new_dict)
                        new_dict = result_dict.copy()

                        for key in random_results[0].keys():
                            if key == 'name':
                                continue
                            new_dict[key] = np.mean([res[key] for res in random_results], axis=0)
                    else:
                        run_analysis_celeba(retrieval_function, k, tol, new_dict, catagories, catagory_column, indistinguisable_values, ['Pale_Skin', 'Male', 'Young'], [['1'], ['1'], ['1']], [['-1'], ['-1'], ['-1']], totals_by_cat)
                    result_dicts.append(new_dict)

df = parse_analysis_celeba(result_dicts, ['Pale_Skin', 'Male', 'Young'])

df.to_pickle("results/celeba-validation-white-nonwhite.pkl")


Starting analysis for method: Baseline...


100%|██████████| 4/4 [00:01<00:00,  3.17it/s]


Starting analysis for method: CDI_Sum_intersectional...


100%|██████████| 4/4 [11:05<00:00, 166.33s/it]


Starting analysis for method: CDI_Min_intersectional...


...

100%|██████████| 4/4 [03:44<00:00, 56.19s/it]


Starting analysis for method: CLIP_intersectional...


100%|██████████| 4/4 [01:15<00:00, 18.91s/it]


Starting analysis for method: DebiasClip...


100%|██████████| 4/4 [00:11<00:00,  2.79s/it]
